**Actividad M1**

Código elaborado por Cristóbal Alberto Escamilla Sada - A00827074

Para la elaboración de este código me apoyé de la documentación de mesa, el repositorio 'mesa-examples' en Github, y del documento Game_of_Life creado por el profesor.

In [ ]:
# Instalación del framework 'mesa'
!pip3 install mesa

In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que puede haber más de un agente por celda elegimos `MultiGrid` que permite más de un objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [ ]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if content.estado_celda == 0:
            grid[x][y] = 0
          elif content.estado_celda == 1:
            grid[x][y] = 1
          else:
            grid[x][y] = 2
    
    return grid

class Aspiradora(Agent):
    '''
    Representa a una aspiradora que limpia el piso
    '''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.estado_celda = 2;
        self.next_position = None

    def step(self):
        neighborhood = self.model.grid.get_neighborhood(
        self.pos,
        moore=True,
        include_center=False)
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        pisos = [obj for obj in this_cell if isinstance(obj, Piso)]

        if pisos[0].estado_celda == 1:
          pisos[0].estado_celda -= 1
          self.next_position = self.pos
          self.model.numCeldasLimpias += 1
        else:
          self.next_position = self.random.choice(neighborhood)
        
        self.model.movimientos += 1

    def advance(self):
        self.model.grid.move_agent(self, self.next_position)

class Piso(Agent):
    '''
    Representa a un cuadro de piso
    '''
    def __init__(self, unique_id, model, estado_celda):
      super().__init__(unique_id, model)
      self.estado_celda = estado_celda
    
class HabitacionModel(Model):
    '''
    Define el modelo de la habitación
    '''
    def __init__(self, width, height, numAspiradoras, numCeldasSucias):
      self.grid = MultiGrid(width, height, True)
      self.schedule = SimultaneousActivation(self)
      self.numAspiradoras = numAspiradoras
      self.numCeldasSucias = numCeldasSucias
      self.numCeldasLimpias = width * height - numCeldasSucias
      self.movimientos = 0

      estados = [0] * width * height
      estadosPos = list(range(width * height))

      contador = 0
      for i in range(numCeldasSucias):
        rand = self.random.choice(estadosPos)
        estados[rand] += 1
        estadosPos.remove(rand)
        contador += 1

      contador = 0
      for (content, x, y) in self.grid.coord_iter():
        p = Piso((x, y), self, estados[contador])
        self.grid.place_agent(p, (x, y))
        self.schedule.add(p)
        contador += 1

      for i in range (self.numAspiradoras):
        a = Aspiradora(i, self)
        self.grid.place_agent(a, (1, 1))
        self.schedule.add(a)

    
    # Aquí definimos con colector para obtener el grid completo.
      self.datacollector = DataCollector(
      model_reporters={"Grid": get_grid})
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()




In [ ]:
# Datos de la habitación:
print("Entrada:")
M = int(input("Tamaño horizontal de la habitación: "))
N = int(input("Tamaño vertical de la habitación: "))

# Número de aspiradoras:
NUM_ASPIRADORAS = int(input("Número de aspiradoras: "))

# Porcentaje de celdas inicialmente sucias:
PORCENTAJE_CELDAS_SUCIAS = float(input("Porcentaje de celdas sucias: "))
NUM_CELDAS_SUCIAS = int(M * N * PORCENTAJE_CELDAS_SUCIAS * 0.01)

# Tiempo máximo de ejecución:
TIEMPO_MAXIMO_EJECUCION = float(input("Tiempo máximo de ejecución (segundos): "))

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = HabitacionModel(M, N, NUM_ASPIRADORAS, NUM_CELDAS_SUCIAS)
while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and modelo.numCeldasLimpias < M * N):
  modelo.step()

tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

print("\nSalida:\nEjecución completa.\nPorcentaje de celdas limpias: " + str(modelo.numCeldasLimpias/(M * N) * 100) + "%")
print("Número movimientos: " + str(modelo.movimientos))
print("Tiempo de ejecución: " + str(tiempo_ejecucion) + " segundos")

Entrada:
Tamaño horizontal de la habitación: 20
Tamaño vertical de la habitación: 20
Número de aspiradoras: 50
Porcentaje de celdas sucias: 50
Tiempo máximo de ejecución (segundos): 0.12

Salida:
Ejecución completa.
Porcentaje de celdas limpias: 100.0%
Número movimientos: 4850
Tiempo de ejecución: 0:00:00.102828 segundos


In [ ]:
all_grid = modelo.datacollector.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
print("Leyenda")
print("Celdas blancas: Representan piso limpio")
print("Celdas grises: Representan piso sucio")
print("Celdas negras: Representan aspiradoras inteligentes")
anim

Leyenda
Celdas blancas: Representan piso limpio
Celdas grises: Representan piso sucio
Celdas negras: Representan aspiradoras inteligentes
